In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install transformers==4.32.0 sentencepiece==0.1.99 accelerate==0.23.0 datasets==2.14.5

In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "pfnet/plamo-13b",
    trust_remote_code=True,
)
model = AutoModelForCausalLM.from_pretrained(
    "pfnet/plamo-13b",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [111]:
model.config

PlamoConfig {
  "_name_or_path": "pfnet/plamo-13b",
  "architectures": [
    "PlamoForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "pfnet/plamo-13b--modeling_plamo.PlamoConfig",
    "AutoModelForCausalLM": "pfnet/plamo-13b--modeling_plamo.PlamoForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 16640,
  "max_position_embeddings": 8192,
  "model_type": "plamo",
  "n_shared_head": 8,
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "num_key_value_heads": 40,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "tokenizer_class": "PlamoTokenizer",
  "torch_dtype": "bfloat16",
  "transformers_version": "4.32.0",
  "use_cache": false,
  "vocab_size": 50432
}

## JCommonsenseQA - 1-shot

PLaMo-13Bの性能を評価してみる。PFNの技術ブログ[1]によると、ベンチマークとしてJCommonsenseQAを用いたとのことなのでここでも同じものを用いる。
評価用に用いられるvalidation splitは1,119件だが、自分の環境（RTX4090 x1）では1時間程度かかるため、ここでは100件をサンプリングして簡易的な評価を行う。

### 評価方法

まずは1-shotの評価を試みる。1-shotの性能評価では、質問と回答の形式を指定した例文をコンテクストとしてプロンプトに与えるのが一般的のようなので、ここでも同じ形式とする。

### 評価指標 

評価指標は[1]によるとnormalized accuracyが使われている。
これは選択肢の順序の分布の不均衡の影響を減らすための指標で、以下のように定義される。

$$\text{norm\_acc} = \frac{1}{C} \sum_{i=1}^{C} \text{acc}_i$$

### 参考資料

- [1] https://tech.preferred.jp/ja/blog/llm-plamo/

### データセットのダウンロード

In [79]:
dataset = load_dataset("leemeng/jcommonsenseqa-v1.1", split="validation")
shuffled_dataset = dataset.shuffle(seed=42)
sampled_dataset = shuffled_dataset.select(range(100))

sampled_dataset

Dataset({
    features: ['q_id', 'question', 'choice0', 'choice1', 'choice2', 'choice3', 'choice4', 'label'],
    num_rows: 100
})

### 質問-回答例のサンプリング

コンテクストとして与える質問と模範回答の例を作成する。モデルの評価にはvalidation splitを利用したので、ここではtrain splitからサンプリングする。

In [ ]:
train_dataset = load_dataset("leemeng/jcommonsenseqa-v1.1", split="train")

In [105]:
display(train_dataset[0])

{'q_id': 0,
 'question': '主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？',
 'choice0': '世界',
 'choice1': '写真集',
 'choice2': '絵本',
 'choice3': '論文',
 'choice4': '図鑑',
 'label': 2}

In [81]:
sample_prompt = """質問: 主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？
choice0: 世界
choice1: 写真集
choice2: 絵本
choice3: 論文
choice4: 図鑑
回答: 絵本"""

### テキストの生成

In [165]:
for i, item in tqdm(enumerate(sampled_dataset), total=sampled_dataset.num_rows):
    text = f"""質問: {item["question"]}\nchoice0: {item["choice0"]}\nchoice1: {item["choice1"]}\nchoice2: {item["choice2"]}\nchoice3: {item["choice3"]}\nchoice4: {item["choice4"]}\n解答: """
    prompt_text = f"### 例 ###\n{sample_prompt}\n\n{text}"
    print(prompt_text)
    break

  0%|          | 0/100 [00:00<?, ?it/s]

### 例 ###
質問: 主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？
choice0: 世界
choice1: 写真集
choice2: 絵本
choice3: 論文
choice4: 図鑑
回答: 絵本

質問: ポストに入れたハガキを送るのは何と言う？
choice0: 郵便
choice1: デスク
choice2: ファクシミリ
choice3: 電子メール
choice4: キャビネット
解答: 


In [91]:
answers = []

for i, item in tqdm(enumerate(sampled_dataset), total=sampled_dataset.num_rows):
    text = f"""質問: {item["question"]}\nchoice0: {item["choice0"]}\nchoice1: {item["choice1"]}\nchoice2: {item["choice2"]}\nchoice3: {item["choice3"]}\nchoice4: {item["choice4"]}\n解答: """
    prompt_text = f"### 例 ###\n{sample_prompt}\n\n{text}"
    prompt = tokenizer(prompt_text, return_tensors="pt").input_ids
    prompt_len = len(prompt[0])
    prompt = prompt.to(model.device)
    generated_tokens = model.generate(
        inputs=prompt,
        max_new_tokens=5,
        do_sample=False,
    )[0]
    generated_text = tokenizer.decode(generated_tokens[prompt_len:])
    answer = generated_text.split("\n")[0]
    answers.append(answer)

  0%|          | 0/100 [00:00<?, ?it/s]

In [92]:
correct_answers = []
for item in sampled_dataset:
    choices = [item[f"choice{i}"] for i in range(5)]
    label = item["label"]
    correct_answers.append(choices[label])

In [96]:
qa_df = sampled_dataset.to_pandas()
qa_df["answer"] = answers
qa_df["correct_answer"] = correct_answers
qa_df.to_csv("jcommonsense_plamo13b_sample100.csv")
qa_df

,q_id,question,choice0,choice1,choice2,choice3,choice4,label,answer,correct_answer
0,9175,ポストに入れたハガキを送るのは何と言う？,郵便,デスク,ファクシミリ,電子メール,キャビネット,0,郵便,郵便
1,9813,全世界の願いは？,平和,人々を惹きつける演説をする,戦争,貿易,テロ,0,平和,平和
2,9081,お正月を別の言い方でなんと言う？,歳末,元旦,お年玉,年末,年賀状,1,お年玉,元旦
3,9623,建物の一番手前にあるのは？,キウイ,食堂,事務所,入り口,村人,3,入り口,入り口
4,9824,身体の自由が利かず動けない状態とは？,元気,疲労,愛,電波,回復,1,疲労,疲労
...,...,...,...,...,...,...,...,...,...,...
95,9508,金属製の小さな容器のことを何という？,瓶,肉屋,缶,八百屋,マーケット,2,瓶,缶
96,9071,バラバラの形をしたピースを繋ぎ合わせて1枚の絵を完成させる物を何という？,ヨーヨー,すごろく,ボードゲーム,トランプ,ジグソーパズル,4,ジグソーパズル,ジグソーパズル
97,9631,よくバッグに使われる素材は？,ハンド,プレゼント,手提げ袋,ミュール,革,4,手提げ袋,革
98,9856,カチカチに凍ってしまうことを何付くという？,凍み付く,気付く,眠りに付く,嘘を付く,居付く,0,凍み付く,凍み付く


### 評価

サンプルしたデータに対しては、約60%のnorm_accが得られた。PFNの公開データ[1]によると53.4%とのことなので、概ね似たスコアが得られている。

In [137]:


def norm_acc(df):
    df = df.copy()
    accs = []
    for label, df_ in df.groupby("label"):
        acc = df_.apply(lambda item: item["answer"] == item["correct_answer"], axis=1).mean()
        accs.append(acc)
    return np.mean(accs), accs

In [142]:
acc, accs = norm_acc(qa_df)
print(f"norm_acc(1-shot): {100 * acc:.1f}%")
print([f"{i}: {acc * 100:.1f}%" for i, acc in enumerate(accs)])


norm_acc(1-shot): 60.7%
['0: 60.0%', '1: 62.5%', '2: 38.1%', '3: 69.6%', '4: 73.3%']


## JCommonsenseQA - 2-shot

コンテクストとして質問と回答の例をもう一例追加し、2-shotの性能を評価する。

In [99]:
train_dataset[1]

{'q_id': 1,
 'question': '未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？',
 'choice0': '浮浪者',
 'choice1': '保護者',
 'choice2': 'お坊さん',
 'choice3': '宗教者',
 'choice4': '預言者',
 'label': 1}

In [101]:
sample_prompt_2 = """質問: 未成年者を監護・教育し，彼らを監督し，彼らの財産上の利益を守る法律上の義務をもつ人は？
choice0: 浮浪者
choice1: 保護者
choice2: お坊さん
choice3: 宗教者
choice4: 預言者
回答: 保護者"""

In [104]:
from tqdm.notebook import tqdm

answers_2shot = []

for i, item in tqdm(enumerate(sampled_dataset), total=sampled_dataset.num_rows):
    text = f"""質問: {item["question"]}\nchoice0: {item["choice0"]}\nchoice1: {item["choice1"]}\nchoice2: {item["choice2"]}\nchoice3: {item["choice3"]}\nchoice4: {item["choice4"]}\n解答: """
    prompt_text = f"### 例 ###\n{sample_prompt}\n\n### 例 ###\n{sample_prompt_2}\n\n{text}"
    prompt = tokenizer(prompt_text, return_tensors="pt").input_ids
    prompt_len = len(prompt[0])
    prompt = prompt.to(model.device)
    generated_tokens = model.generate(
        inputs=prompt,
        max_new_tokens=5,
        do_sample=False,
    )[0]
    generated_text = tokenizer.decode(generated_tokens[prompt_len:])
    answer = generated_text.split("\n")[0]
    answers_2shot.append(answer)

  0%|          | 0/100 [00:00<?, ?it/s]

In [107]:
qa_df2 = sampled_dataset.to_pandas()
qa_df2["answer"] = answers_2shot
qa_df2["correct_answer"] = correct_answers
qa_df2.to_csv("jcommonsense_plamo13b_sample100_2shot.csv")
qa_df2

,q_id,question,choice0,choice1,choice2,choice3,choice4,label,answer,correct_answer
0,9175,ポストに入れたハガキを送るのは何と言う？,郵便,デスク,ファクシミリ,電子メール,キャビネット,0,郵便,郵便
1,9813,全世界の願いは？,平和,人々を惹きつける演説をする,戦争,貿易,テロ,0,平和,平和
2,9081,お正月を別の言い方でなんと言う？,歳末,元旦,お年玉,年末,年賀状,1,お年玉,元旦
3,9623,建物の一番手前にあるのは？,キウイ,食堂,事務所,入り口,村人,3,入り口,入り口
4,9824,身体の自由が利かず動けない状態とは？,元気,疲労,愛,電波,回復,1,疲労,疲労
...,...,...,...,...,...,...,...,...,...,...
95,9508,金属製の小さな容器のことを何という？,瓶,肉屋,缶,八百屋,マーケット,2,瓶,缶
96,9071,バラバラの形をしたピースを繋ぎ合わせて1枚の絵を完成させる物を何という？,ヨーヨー,すごろく,ボードゲーム,トランプ,ジグソーパズル,4,トランプ,ジグソーパズル
97,9631,よくバッグに使われる素材は？,ハンド,プレゼント,手提げ袋,ミュール,革,4,手提げ袋,革
98,9856,カチカチに凍ってしまうことを何付くという？,凍み付く,気付く,眠りに付く,嘘を付く,居付く,0,居付く,凍み付く


In [126]:
print(norm_acc(qa_df2))

0.589712215320911


1-shotの場合よりnorm_accはやや減少したが、サンプル数が100であることを考えると概ね同程度の性能である。
サンプル数が少ないのでちゃんとした評価にはならないが、例文の数を増やしても回答の性能には大きく寄与しないようである。

## Appendix

### A. 誤答について

モデルが誤答した質問と回答について以下に示す。

In [166]:
for _, item in qa_df.query("answer != correct_answer").iterrows():
    print(
        f"""問題: {item["question"]}
選択肢: {item["choice0"]}, {item["choice1"]}, {item["choice2"]}, {item["choice3"]}, {item["choice4"]}
正答: {item["correct_answer"]}
モデルの回答: {item["answer"]}
"""
    )

問題: お正月を別の言い方でなんと言う？
選択肢: 歳末, 元旦, お年玉, 年末, 年賀状
正答: 元旦
モデルの回答: お年玉

問題: イベントは？
選択肢: お祭り, 銀行, コンビニ, 駄菓子屋, 病院
正答: お祭り
モデルの回答: 病院

問題: 食卓にかけるものは？
選択肢: スチュワーデス, テーブルクロス, 醤油, シートベルト, 塩
正答: テーブルクロス
モデルの回答: 醤油

問題: 何かに通るためにチャレンジする事とは？
選択肢: 調べる, 受験, 卯, 幼馴染, 合格する
正答: 受験
モデルの回答: 調べる

問題: 授業内容の理解を図るイベントは？
選択肢: 学校に通う, 自動車免許, 会議, 期末試験を受ける, 宿題
正答: 期末試験を受ける
モデルの回答: 会議

問題: 心の拠り所になるものは？
選択肢: 宗教, 職場, キャンドル, 家, 尖塔
正答: 宗教
モデルの回答: 家

問題: 小さい島の集まりは？
選択肢: 信号機, 天津飯, マーシャル諸島, 東南アジア, 関東
正答: マーシャル諸島
モデルの回答: 信号機

問題: ワインを保管する場所は？
選択肢: ワインセラー, 扇風機, アイスボックス, 冷凍庫, 乾燥機
正答: ワインセラー
モデルの回答: 冷凍庫

問題: アジアとヨーロッパの架け橋と呼ばれる国は？
選択肢: トルコ, エジプト, イスラエル, 砂漠, イラン
正答: トルコ
モデルの回答: イラン

問題: 文字を送る郵便は？
選択肢: FAX, 受付係, 便箋, 絵はがき, 段ボール
正答: 便箋
モデルの回答: 絵はがき

問題: 星などを背景にすることによって、黒く浮かび上がって見えるのは？
選択肢: 暗黒星雲, 翼, 暗黒面, 暗黒時代, 受ける
正答: 暗黒星雲
モデルの回答: 暗黒面

問題: 同じ学校や職場の人が一緒に暮らすところは？
選択肢: 画房, マンション, 寮, アパート, 引出
正答: 寮
モデルの回答: アパート

問題: 「ざんごう」と読むのは？
選択肢: 北東, 向こう, 南東, 塹壕, 中東
正答: 塹壕
モデルの回答: 南東

問題: 学校へは通学せずに家庭で学習するのは？
選択肢: 動画, 基礎勉強, 包丁, ホームスクール, 冷蔵庫
正答: ホームスクール